In [2]:
from pathlib import Path
import sys, os, numpy as np

PROJECT_ROOT = Path.cwd().parents[1]
sys.path.append(str(PROJECT_ROOT))

price_from_ytm = None
ytm_from_price = None

try:
    import ytm_analysis as 
    # Try likely names; adjust if needed
    price_from_ytm = getattr(ya, "price_from_ytm", None) or getattr(ya, "bond_price", None) or getattr(ya, "price_bond", None)
    ytm_from_price = getattr(ya, "ytm_from_price", None) or getattr(ya, "yield_to_maturity", None) or getattr(ya, "calculate_ytm", None) or getattr(ya, "calc_ytm", None)
except Exception:
    pass

if price_from_ytm is None or ytm_from_price is None:
    # Fallback implementations
    def price_from_ytm(face, coupon_rate, ytm, maturity, freq=2):
        n = int(round(maturity*freq))
        c = face * coupon_rate / freq
        r = ytm / freq
        t = np.arange(1, n+1)
        return (c / (1+r)**t).sum() + face / (1+r)**n

    def ytm_from_price(face, coupon_rate, price, maturity, freq=2, guess=0.05, tol=1e-10, maxiter=100):
        y = guess
        for _ in range(maxiter):
            n = int(round(maturity*freq))
            c = face * coupon_rate / freq
            r = y / freq
            t = np.arange(1, n+1)
            pv = (c / (1+r)**t).sum() + face / (1+r)**n
            f  = pv - price
            # dP/dy = (dP/dr)*(1/freq); dP/dr = -Σ t*c/(1+r)^(t+1) - n*F/(1+r)^(n+1)
            dP_dr = - ( (t*c / (1+r)**(t+1)).sum() + (n*face) / (1+r)**(n+1) )
            dP_dy = dP_dr / freq
            y_new = y - f / dP_dy
            if abs(y_new - y) < tol:
                return y_new
            y = y_new
        return y

In [3]:
# Example: 5y 4.5% coupon, 4.35% YTM, face 100, semiannual
price = price_from_ytm(face=100, coupon_rate=0.045, ytm=0.0435, maturity=5, freq=2)
ytm   = ytm_from_price(face=100, coupon_rate=0.045, price=price, maturity=5, freq=2)

print(f"Price: {price:.2f} | Backed-out YTM: {ytm*100:.3f}%")

out_dir = PROJECT_ROOT / "outputs" / "03_YTM"
os.makedirs(out_dir, exist_ok=True)
with open(out_dir / "example.txt", "w") as f:
    f.write(f"Price={price:.2f}, YTM={ytm:.5f}")
print("✔ Saved:", out_dir / "example.txt")

Price: 100.67 | Backed-out YTM: 4.350%
✔ Saved: /Users/katherinecohen/Documents/FixedIncomePortfolio/outputs/03_YTM/example.txt
